In [1]:
import os
os.chdir("../")

In [2]:
import pandas as pd
import json
import sys
sys.path.append('./src')
from data import load_data

def generate_monthly_data(df,groupby_col, date_col='date'):
    result_dfs = []
    for well, data in df.groupby(groupby_col):
        data = data.set_index(date_col)
        idx = pd.date_range(data.index.min(), data.index.max(), freq='MS')
        data = data.reindex(idx)
        data = data.reset_index()
        data[groupby_col] = well
        result_dfs.append(data)
    result_df = pd.concat(result_dfs)
    return result_df

config_file = "./config.json"
with open(config_file, 'r') as f:
    config = json.load(f)
train_data_path = config["train_data_path"]
df = load_data(train_data_path) 
X = df.drop(columns='cluster')
y = df.cluster
df = (X.explode(['date','value'])
      .assign(date = lambda df: pd.to_datetime(df['date']))
      .pipe(generate_monthly_data,'well', 'date')
      )

In [53]:
import pandas as pd
import json
import sys
sys.path.append('./src')
from data import load_data
def round_to_nearest_multiple(num, multiple):
    return round(num / multiple) * multiple

def calculate_relative_time(df, groupby_col, date_col='date', rounded = True):
    min_dates = df.groupby(groupby_col)[date_col].min()
    df['relative_time'] = df.apply(lambda row: (row[date_col] - min_dates[row[groupby_col]]).days, axis=1)
    if rounded:
        df['relative_time']  =  round_to_nearest_multiple(df['relative_time'], 30).astype(int)
    return df

config_file = "./config.json"
with open(config_file, 'r') as f:
    config = json.load(f)
train_data_path = config["train_data_path"]
df = load_data(train_data_path) 
X = df.drop(columns='cluster')
y = df.cluster
df = (X.explode(['date','value'])
      .assign(date = lambda df: pd.to_datetime(df['date']))
      .pipe(calculate_relative_time, 'well', 'date')
      )
df

,well,date,value,relative_time
0,well_9,1977-01-01,0.939394,0
0,well_9,1977-02-01,0.965517,30
0,well_9,1977-03-01,0.0,60
0,well_9,1977-04-01,0.0,90
0,well_9,1977-05-01,0.0,120
...,...,...,...,...
369,well_575,2017-10-01,12.052015,3420
369,well_575,2017-11-01,14.237062,3450
369,well_575,2017-12-01,17.529553,3480
369,well_575,2018-01-01,17.726489,3510


In [61]:
col = "relative_time"
groupby_col = 'well'
idx = list(range(df[col].min(),df[col].max()+30,30))
result_dfs = []
df_grouped = df.groupby(groupby_col)
well='well_9'
data = df_grouped.get_group(well)
data = data.set_index(col)
data.reindex(idx)

ValueError: cannot reindex on an axis with duplicate labels

In [54]:
col = "relative_time"
groupby_col = 'well'
idx = list(range(df[col].min(),df[col].max()+30,30))
result_dfs = []
for well, data in df.groupby(groupby_col):
    data = data.set_index(col)
    data = data.reindex(idx)
    data = data.reset_index()
    data[groupby_col] = well
    result_dfs.append(data)
result_df = pd.concat(result_dfs)

ValueError: cannot reindex on an axis with duplicate labels